In [1]:
%display latex

In [2]:
p = 3
N = 2

F = GF(p^N, 'x')
x = F.gen()

In [3]:
basis = [F(1), x + 1] # almost self-dual

def toInt(k):
    return list(F).index(k)

def components(k):
    return vector([(k * b).trace() for b in basis])

omega = exp(2 * pi * I / p)
chi = lambda k: omega^int(k.trace())

In [4]:
for bj in basis:
    for bk in basis:
        print(f'{bj}\t {bk} \t-> {(bj * bk).trace()}')

1	 1 	-> 2
1	 x + 1 	-> 0
x + 1	 1 	-> 0
x + 1	 x + 1 	-> 1


In [5]:
def qj(j):
    return (F(2) - F(1)) * int(j == 0) + F(1)

In [6]:
# verify that it is almost self-dual
for i, bi in enumerate(basis):
    for j, bj in enumerate(basis):
        lhs = (bi * bj).trace()
        rhs = qj(j) * F(int(i == j))
        if lhs != rhs:
            raise Exception(bi, bj, lhs, rhs)

In [7]:
# is the char additive?
for j in F:
    for k in F:
        if chi(j) * chi(k) != chi(j + k):
            raise Exception

In [8]:
# factorization of the character 
for a in F:
    for b in F:
        ais = components(a)
        bis = components(b)
        lhs = chi(a * b)
        rhs = 1
        for i in range(N):
            rhs *= omega^int(ais[i] * bis[i] * qj(i))
        if lhs != rhs:
            print(f'{lhs}\t {rhs}')

$$\Phi(\alpha,\beta) = \chi\left(-2^{-1} \alpha \beta\right)$$
$$\chi(\alpha\beta) = \prod_i^N \omega^{\alpha_i \beta_i q_j}$$

In [9]:
def Phi(a, b):
    return chi(-F(2)^-1 * a * b)

In [10]:
def DPS(curve):
    ps = zero_matrix(SR, p^N, p^N)
    ps[0,:] = 1
    for mu in F:
        for j, a in enumerate(F):
            b = curve(mu)(a)
            ps[j, toInt(b)] = Phi(a, b)
    return ps

In [11]:
def ray(mu):
    return lambda k: mu * k

In [12]:
ps = DPS(ray)
ps

[                      1                       1                       1                       1                       1                       1                       1                       1                       1]
[                      1                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2]
[                      1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1]
[                      1     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2]
[                      1 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2]
[                      1                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2]
[                      1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1]
[                      1 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2]
[                      1     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2]

In [176]:
# w = var('omega')

# ps = DPS(ray)
# ps

[      1       1       1       1       1       1       1       1       1]
[      1       1   omega   omega omega^2       1 omega^2 omega^2   omega]
[      1   omega   omega omega^2       1 omega^2 omega^2   omega       1]
[      1   omega omega^2       1 omega^2 omega^2   omega       1   omega]
[      1 omega^2       1 omega^2 omega^2   omega       1   omega   omega]
[      1       1 omega^2 omega^2   omega       1   omega   omega omega^2]
[      1 omega^2 omega^2   omega       1   omega   omega omega^2       1]
[      1 omega^2   omega       1   omega   omega omega^2       1 omega^2]
[      1   omega       1   omega   omega omega^2       1 omega^2 omega^2]

In [13]:
# verify that the phase satisfies the recurrence relation
def VerifyRecRel(curve=ray):
    for mu in F:
        for t1 in F:
            for t2 in F:
                lhs = Phi(t1, curve(mu)(t1)) * Phi(t2, curve(mu)(t2))
                rhs = Phi(t1 + t2, curve(mu)(t1 + t2)) * chi(t2 * curve(mu)(t1))
                if lhs != rhs:
                    raise Exception
    return True

In [14]:
VerifyRecRel(ray)

True

In [15]:
-F(2)^-1

1

$$\Phi(\alpha, \beta) = \prod_i^N \omega^{-2^{-1}\alpha_i \beta_i q_j}$$

In [16]:
# Test factorization of the phase not just the character
for a in F:
    for b in F:
        ais = components(a)
        bis = components(b)
        lhs = Phi(a, b)
        rhs = 1
        for i in range(N):
            rhs *= omega^int(-F(2)^-1 * ais[i] * bis[i] * qj(i))
        if lhs != rhs:
            print(f'{lhs}\t {rhs}')

In [17]:
w = var('omega')

In [18]:
dps_1comp = zero_matrix(SR, p^N, p^N)
for i, a in enumerate(F):
    for j, b in enumerate(F):
        a1 = components(a)[0]
        b1 = components(b)[0]
        dps_1comp[i,j] = omega^int(-F(2)^-1 * a1 * b1 * qj(0))
dps_1comp

[      1       1       1       1       1       1       1       1       1]
[      1 omega^2       1 omega^2 omega^2   omega       1   omega   omega]
[      1       1       1       1       1       1       1       1       1]
[      1 omega^2       1 omega^2 omega^2   omega       1   omega   omega]
[      1 omega^2       1 omega^2 omega^2   omega       1   omega   omega]
[      1   omega       1   omega   omega omega^2       1 omega^2 omega^2]
[      1       1       1       1       1       1       1       1       1]
[      1   omega       1   omega   omega omega^2       1 omega^2 omega^2]
[      1   omega       1   omega   omega omega^2       1 omega^2 omega^2]

In [19]:
dps_2comp = zero_matrix(SR, p^N, p^N)
for i, a in enumerate(F):
    for j, b in enumerate(F):
        a2 = components(a)[1]
        b2 = components(b)[1]
        dps_2comp[i,j] = omega^int(-F(2)^-1 * a2 * b2 * qj(1))
dps_2comp

[      1       1       1       1       1       1       1       1       1]
[      1   omega   omega omega^2       1 omega^2 omega^2   omega       1]
[      1   omega   omega omega^2       1 omega^2 omega^2   omega       1]
[      1 omega^2 omega^2   omega       1   omega   omega omega^2       1]
[      1       1       1       1       1       1       1       1       1]
[      1 omega^2 omega^2   omega       1   omega   omega omega^2       1]
[      1 omega^2 omega^2   omega       1   omega   omega omega^2       1]
[      1   omega   omega omega^2       1 omega^2 omega^2   omega       1]
[      1       1       1       1       1       1       1       1       1]

In [20]:
ps_prod = dps_1comp.elementwise_product(dps_2comp)
ps_prod

[      1       1       1       1       1       1       1       1       1]
[      1 omega^3   omega omega^4 omega^2 omega^3 omega^2 omega^2   omega]
[      1   omega   omega omega^2       1 omega^2 omega^2   omega       1]
[      1 omega^4 omega^2 omega^3 omega^2 omega^2   omega omega^3   omega]
[      1 omega^2       1 omega^2 omega^2   omega       1   omega   omega]
[      1 omega^3 omega^2 omega^2   omega omega^3   omega omega^4 omega^2]
[      1 omega^2 omega^2   omega       1   omega   omega omega^2       1]
[      1 omega^2   omega omega^3   omega omega^4 omega^2 omega^3 omega^2]
[      1   omega       1   omega   omega omega^2       1 omega^2 omega^2]

In [21]:
for j in range(9):
    for k in range(9):
        if ps_prod[j,k] == w^3:
            ps_prod[j,k] = 1
        if ps_prod[j,k] == w^4:
            ps_prod[j,k] = w

In [22]:
ps_prod

[      1       1       1       1       1       1       1       1       1]
[      1       1   omega   omega omega^2       1 omega^2 omega^2   omega]
[      1   omega   omega omega^2       1 omega^2 omega^2   omega       1]
[      1   omega omega^2       1 omega^2 omega^2   omega       1   omega]
[      1 omega^2       1 omega^2 omega^2   omega       1   omega   omega]
[      1       1 omega^2 omega^2   omega       1   omega   omega omega^2]
[      1 omega^2 omega^2   omega       1   omega   omega omega^2       1]
[      1 omega^2   omega       1   omega   omega omega^2       1 omega^2]
[      1   omega       1   omega   omega omega^2       1 omega^2 omega^2]

In [24]:
block1 = zero_matrix(SR, p, p)
for i, a in [F(0), F(1), F(2)]:
    for j, b in [F(0), F(1), F(2)]:
        block1[i,j] = Phi(a, b)
block1

[1 1 1]
[1 1 1]
[1 1 1]

Let's do a quick test with three qutrits. The phase space has to be the same. Is this the source of the factorization of the kernel?

In [12]:
p = 3
N = 3

F = GF(p^N, 'x')
x = F.gen()

def toInt(k):
    return list(F).index(k)

# omega = exp(2 * pi * I / p)
w = var('omega')
chi = lambda k: omega^int(k.trace())

In [13]:
def Phi(a, b):
    return chi(-F(2)^-1 * a * b)

In [14]:
def DPS(curve):
    ps = zero_matrix(SR, p^N, p^N)
    ps[0,:] = 1
    for mu in F:
        for j, a in enumerate(F):
            b = curve(mu)(a)
            ps[j, toInt(b)] = Phi(a, b)
    return ps

In [24]:
def ray(mu):
    return lambda k: mu * k

# Albert's twisted field
def albert(mu):
    return lambda k: mu * k^9 + mu^3 * k^3

In [25]:
DPS(ray)

27 x 27 dense matrix over Symbolic Ring (use the '.str()' method to see the entries)

In [26]:
DPS(albert)

27 x 27 dense matrix over Symbolic Ring (use the '.str()' method to see the entries)

In [27]:
DPS(ray) == DPS(albert)

True